# Regressao - Base Automóveis

Base de dados: https://archive.ics.uci.edu/dataset/53/iris

## Importando bibliotecas

In [1]:
!pip install -q tensorflow==2.16.1

In [2]:
# Importacao desta lib para desativar erro no TensorFlow
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import pandas as pd
import tensorflow as tf
import sklearn

2024-11-27 14:34:35.602230: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-27 14:34:35.836048: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-27 14:34:36.598864: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 14:34:38.515579: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
pd.__version__,tf.__version__,sklearn.__version__

('2.2.2', '2.16.1', '1.4.2')

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

## Importando base de dados

In [6]:
base = pd.read_csv('autos.csv', encoding='ISO-8859-1')

In [7]:
base.head(5)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


## Tratando base de dados

### Apagando dados não relevantes

In [8]:
base = base.drop('dateCrawled',axis=1)
base = base.drop('dateCreated',axis=1)
base = base.drop('nrOfPictures',axis=1)
base = base.drop('lastSeen',axis=1)
base = base.drop('postalCode',axis=1)

In [9]:
base.head(5)

,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
1,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein


In [10]:
base.shape

(371528, 15)

In [11]:
base['name'].value_counts()

name
Ford_Fiesta                                              657
BMW_318i                                                 627
Opel_Corsa                                               622
Volkswagen_Golf_1.4                                      603
BMW_316i                                                 523
                                                        ... 
Audi_A4_Avant_Klima_Gruene_Plakette_TÜV_&AU_NEU_XENON      1
Renault_clio_in_gold_450VB_!!                              1
Fiat_Doblo_1.6_Multijet                                    1
Renault_Laguna_1                                           1
BMW_M135i_vollausgestattet_NP_52.720____Euro               1
Name: count, Length: 233531, dtype: int64

Como há 233531 itens com total de 1 no name, podemos pensar que este name seja o titulo do anuncio, e sendo assim, o algoritmo não irá se adaptar bem. Podemos excluir esta coluna, e se referenciar à coluna Brand

In [12]:
base = base.drop('name',axis=1)

In [13]:
base['seller'].value_counts()

seller
privat        371525
gewerblich         3
Name: count, dtype: int64

Como praticamente quase todos registros são vendedores privados, esta coluna é irrelevante para o algoritmo

In [14]:
base = base.drop('seller',axis=1)

In [15]:
base['offerType'].value_counts()

offerType
Angebot    371516
Gesuch         12
Name: count, dtype: int64

Como praticamente quase todos registros são vendedores privados, esta coluna é irrelevante para o algoritmo

In [16]:
base = base.drop('offerType',axis=1)

In [17]:
base

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
371524,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN


### Analise de valores inconsistentes

In [18]:
base.loc[base['price'] <= 10]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
7,0,test,limousine,1980,manuell,50,andere,40000,7,benzin,volkswagen,nein
40,0,test,NaN,1990,NaN,0,corsa,150000,1,benzin,opel,NaN
60,1,control,suv,1994,manuell,286,NaN,150000,11,NaN,sonstige_autos,NaN
91,1,control,limousine,1995,manuell,113,e_klasse,150000,4,diesel,mercedes_benz,nein
115,0,test,NaN,2017,manuell,0,golf,5000,12,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371356,0,control,NaN,2000,manuell,65,corsa,150000,0,NaN,opel,ja
371392,0,test,kleinwagen,2002,manuell,60,fiesta,150000,3,benzin,ford,NaN
371402,0,control,kleinwagen,1999,manuell,53,swift,150000,3,benzin,suzuki,NaN
371431,0,control,kleinwagen,1999,manuell,37,arosa,150000,7,benzin,seat,ja


Os prices acima podem ter sido extraidos de forma equivocada, logo podemos substituir estes registros pela média de prices geral. **Porém** como são mais de 12k de registros, isso pode não ser bom, o ideal é deleta-los. **Valores abaixo de 10**

In [19]:
base['price'].mean()

17295.14186548524

In [20]:
base.shape

(371528, 12)

In [21]:
base = base[base['price'] > 10]

In [22]:
base.shape

(359410, 12)

In [23]:
base.loc[base['price'] > 350000]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1846,579000,control,coupe,1980,manuell,277,andere,20000,12,benzin,bmw,nein
10649,420000,control,coupe,2004,manuell,483,911,50000,4,benzin,porsche,nein
14663,11111111,control,coupe,2003,manuell,64,polo,150000,2,benzin,volkswagen,NaN
16889,1000000,control,kombi,1998,NaN,0,mondeo,150000,0,benzin,ford,ja
20143,1250000,test,coupe,2016,manuell,500,911,5000,3,benzin,porsche,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
364171,3890000,test,coupe,2006,NaN,799,NaN,5000,7,NaN,sonstige_autos,nein
365461,599000,control,coupe,1980,manuell,377,andere,5000,3,benzin,bmw,nein
366653,99999999,control,cabrio,1996,manuell,192,3er,150000,0,NaN,bmw,NaN
366861,3895000,test,coupe,2006,NaN,799,NaN,5000,4,benzin,sonstige_autos,nein


Os prices acima podem ter sido extraidos de forma equivocada, logo podemos substituir estes registros pela média de prices geral. **Porém** como são mais de 12k de registros, isso pode não ser bom, o ideal é deleta-los. **Valores acima de 350k**

In [24]:
base = base.loc[base['price'] < 350000]

In [25]:
base.shape

(359291, 12)

### Valores faltantes

In [26]:
base.loc[pd.isnull(base['vehicleType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
22,2900,test,NaN,2018,manuell,90,meriva,150000,5,benzin,opel,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
31,899,control,NaN,2016,manuell,60,clio,150000,6,benzin,renault,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


Em valores númericos faltantes, é normal ser substituido os faltantes pela média. No caso de dados categóricos, do vehicleType, pode-se preencher os NaN com a moda

In [27]:
base['vehicleType'].value_counts()

vehicleType
limousine     93614
kleinwagen    78014
kombi         65921
bus           29699
cabrio        22509
coupe         18386
suv           14477
andere         3125
Name: count, dtype: int64

In [28]:
base['vehicleType'].mode()

0    limousine
Name: vehicleType, dtype: object

In [29]:
base.loc[pd.isnull(base['gearbox'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
15,450,test,kleinwagen,1910,NaN,0,ka,5000,0,benzin,ford,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
32,245,test,limousine,1994,NaN,0,golf,150000,2,benzin,volkswagen,nein
37,1500,test,NaN,2016,NaN,0,kangoo,150000,1,diesel,renault,nein
70,1200,test,coupe,2001,NaN,0,astra,150000,0,NaN,opel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371443,3300,control,kombi,2006,NaN,0,touran,150000,7,diesel,volkswagen,NaN
371460,3500,control,NaN,1995,NaN,0,polo,150000,0,NaN,volkswagen,NaN
371486,350,control,kleinwagen,1996,NaN,65,punto,150000,0,NaN,fiat,NaN
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN


In [30]:
base['gearbox'].value_counts()

gearbox
manuell      266547
automatik     75508
Name: count, dtype: int64

In [31]:
base.loc[pd.isnull(base['model'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
83,350,control,kleinwagen,1997,manuell,54,NaN,150000,3,NaN,fiat,ja
139,1450,control,limousine,1992,manuell,136,NaN,150000,0,NaN,audi,nein
156,6799,control,kleinwagen,2009,NaN,60,NaN,20000,5,benzin,volkswagen,nein
165,500,control,kleinwagen,1999,manuell,0,NaN,150000,0,benzin,renault,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371399,560,control,kleinwagen,2001,automatik,170,NaN,90000,0,benzin,fiat,ja
371476,9400,control,kombi,2007,manuell,200,NaN,150000,4,diesel,sonstige_autos,ja
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein


In [32]:
base['model'].value_counts()

model
golf               28989
andere             25560
3er                19905
polo               12604
corsa              12149
                   ...  
serie_2                8
rangerover             6
serie_3                3
serie_1                1
discovery_sport        1
Name: count, Length: 251, dtype: int64

In [33]:
base.loc[pd.isnull(base['fuelType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
13,2500,control,kombi,2004,manuell,131,passat,150000,2,NaN,volkswagen,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
36,1600,control,andere,1991,manuell,75,kadett,70000,0,NaN,opel,NaN
41,7500,control,limousine,2002,automatik,306,e_klasse,150000,4,NaN,mercedes_benz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371496,3850,test,cabrio,2006,manuell,108,2_reihe,125000,2,NaN,peugeot,nein
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [34]:
base['fuelType'].value_counts()

fuelType
benzin     217582
diesel     106002
lpg          5222
cng           557
hybrid        271
andere        165
elektro       101
Name: count, dtype: int64

In [35]:
base.loc[pd.isnull(base['notRepairedDamage'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
8,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
12,999,control,kombi,1995,manuell,115,passat,150000,11,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371507,5999,test,kombi,2005,manuell,140,a4,150000,4,diesel,audi,NaN
371514,999,control,cabrio,2000,manuell,95,megane,150000,4,benzin,renault,NaN
371515,1690,test,kombi,2004,manuell,55,fabia,150000,4,benzin,skoda,NaN
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN


In [36]:
base['notRepairedDamage'].value_counts()

notRepairedDamage
nein    259301
ja       34004
Name: count, dtype: int64

In [37]:
valores = {
    'vehicleType': 'limousine',
    'gearbox':'manuell',
    'model':'golf',
    'fuelType':'benzin',
    'notRepairedDamage':'nein'
}

In [38]:
base = base.fillna(value = valores)

In [39]:
base

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,limousine,1993,manuell,0,golf,150000,0,benzin,volkswagen,nein
1,18300,test,coupe,2011,manuell,190,golf,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,nein
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2200,test,limousine,2005,manuell,0,golf,20000,1,benzin,sonstige_autos,nein
371524,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,nein


In [40]:
base.isnull()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,False,False,False,False,False,False,False,False,False,False,False,False
371524,False,False,False,False,False,False,False,False,False,False,False,False
371525,False,False,False,False,False,False,False,False,False,False,False,False
371526,False,False,False,False,False,False,False,False,False,False,False,False


In [41]:
base.isnull().sum()

price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dtype: int64

In [42]:
base.columns

Index(['price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox',
       'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
       'brand', 'notRepairedDamage'],
      dtype='object')

In [43]:
X = base.iloc[:,1:12].values
X

array([['test', 'limousine', 1993, ..., 'benzin', 'volkswagen', 'nein'],
       ['test', 'coupe', 2011, ..., 'diesel', 'audi', 'ja'],
       ['test', 'suv', 2004, ..., 'diesel', 'jeep', 'nein'],
       ...,
       ['test', 'bus', 1996, ..., 'diesel', 'volkswagen', 'nein'],
       ['test', 'kombi', 2002, ..., 'diesel', 'volkswagen', 'nein'],
       ['control', 'limousine', 2013, ..., 'benzin', 'bmw', 'nein']],
      dtype=object)

In [44]:
y = base.iloc[:,0].values
y

array([  480, 18300,  9800, ...,  9200,  3400, 28990])

### Atributos categóricos

In [45]:
base['brand'].value_counts()

brand
volkswagen        76755
bmw               38982
opel              38557
mercedes_benz     34498
audi              31884
ford              24761
renault           17367
peugeot           10777
fiat               9339
seat               6815
skoda              5572
mazda              5529
smart              5159
citroen            5038
nissan             4893
toyota             4616
hyundai            3585
mini               3354
sonstige_autos     3331
volvo              3229
mitsubishi         2956
honda              2726
kia                2494
suzuki             2274
alfa_romeo         2250
porsche            2154
chevrolet          1791
chrysler           1373
dacia               889
jeep                784
daihatsu            774
land_rover          759
subaru              746
jaguar              603
daewoo              530
saab                516
trabant             489
lancia              467
rover               462
lada                213
Name: count, dtype: int64

In [46]:
onehotencoder = ColumnTransformer(transformers=[("OneHot",OneHotEncoder(),[0,1,3,5,8,9,10])],remainder='passthrough') # Neste array são passados os campos categoricos

In [47]:
X = onehotencoder.fit_transform(X).toarray()
X

array([[0.00e+00, 1.00e+00, 0.00e+00, ..., 0.00e+00, 1.50e+05, 0.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.90e+02, 1.25e+05, 5.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.63e+02, 1.25e+05, 8.00e+00],
       ...,
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.02e+02, 1.50e+05, 3.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.00e+02, 1.50e+05, 6.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, ..., 3.20e+02, 5.00e+04, 8.00e+00]])

In [48]:
base.shape

(359291, 12)

In [49]:
X.shape

(359291, 316)

## Estrutura da rede neural

In [50]:
# Para calcular a quantidade de neuronios na camada dense
# é a soma de entradas + saidas / 2
(316 + 1) / 2 

158.5

In [51]:
regressor = Sequential([
    tf.keras.layers.InputLayer(shape=(316,)),
    tf.keras.layers.Dense(units = 158, activation = 'relu'),
    tf.keras.layers.Dense(units = 158, activation = 'relu'),
    tf.keras.layers.Dense(units = 1, activation = 'linear'),
])

In [52]:
regressor.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 158)            │        50,086 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 158)            │        25,122 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           159 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,367 (294.40 KB)

 Trainable params: 75,367 (294.40 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
regressor.compile(loss='mean_absolute_error', optimizer = 'adam', metrics = ['mean_absolute_error'])

In [54]:
regressor.fit(X,y, batch_size = 300, epochs=100)

Epoch 1/100


2024-11-26 18:01:01.118981: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 454143824 exceeds 10% of free system memory.


1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 4216.3311 - mean_absolute_error: 4216.3311
Epoch 2/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3487.4089 - mean_absolute_error: 3487.4089
Epoch 3/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3326.4631 - mean_absolute_error: 3326.4631
Epoch 4/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3011.7192 - mean_absolute_error: 3011.7192
Epoch 5/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2839.0767 - mean_absolute_error: 2839.0767
Epoch 6/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2782.4741 - mean_absolute_error: 2782.4741
Epoch 7/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2736.6682 - mean_absolute_error: 2736.6682
Epoch 8/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 2698.7236 - mean_absolute_error: 2698.7236
Epoch 9/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2660.9690 - mean_absolute_error: 2660.9690
Epoch 10/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - 

In [55]:
X.shape

(359291, 316)

In [56]:
previsoes = regressor.predict(X)

  115/11228 ━━━━━━━━━━━━━━━━━━━━ 9s 889us/step 

2024-11-26 18:08:19.142956: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 454143824 exceeds 10% of free system memory.


11228/11228 ━━━━━━━━━━━━━━━━━━━━ 9s 794us/step


In [57]:
previsoes

array([[ 1626.1188],
       [11563.357 ],
       [14156.337 ],
       ...,
       [ 6581.846 ],
       [ 3421.869 ],
       [27998.312 ]], dtype=float32)

In [58]:
y

array([  480, 18300,  9800, ...,  9200,  3400, 28990])

In [59]:
y.mean()

5916.833945186492

In [60]:
previsoes.mean()

5631.715

## Validacao Cruzada

In [50]:
!pip install -q scikeras

In [51]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [52]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.2.2', '2.16.1', '1.4.2', '0.13.0')

In [53]:
import time
from scikeras.wrappers import KerasRegressor
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

In [65]:
inicio = time.time()
inicio

1732655345.091065

In [66]:
X.shape

(359291, 316)

In [67]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
        tf.keras.layers.InputLayer(shape=(316,)),
        tf.keras.layers.Dense(units=158,activation='relu'),
        tf.keras.layers.Dense(units=158,activation='relu'),
        tf.keras.layers.Dense(units=1,activation='linear'),
    ])
    regressor.compile(loss='mean_absolute_error',optimizer='adam',metrics=['mean_absolute_error'])
    return regressor

In [68]:
regressor = KerasRegressor(model = criar_rede, epochs=100, batch_size=300)

In [69]:
resultados = cross_val_score(estimator=regressor,X=X,y=y, cv=5, scoring='neg_mean_absolute_error')

Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 4253.5762 - mean_absolute_error: 4253.5762
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3524.9006 - mean_absolute_error: 3524.9006
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3425.5479 - mean_absolute_error: 3425.5479
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3285.8120 - mean_absolute_error: 3285.8120
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3015.8945 - mean_absolute_error: 3015.8945
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2902.7607 - mean_absolute_error: 2902.7607
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 2794.2285 - mean_absolute_error: 2794.2285
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2772.8000 - mean_absolute_error: 2772.8000
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2743.8254 - mean_absolute_error: 2743.8254
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 27

In [70]:
fim = time.time()
fim

1732657262.174857

In [71]:
fim - inicio

1917.0837919712067

In [72]:
resultados

array([-2191.7495501 , -2327.76335182, -2699.75930969, -2334.4184007 ,
       -2267.72852475])

In [73]:
abs(resultados.mean())

2364.283827411358

In [74]:
resultados.std()

175.41977732431454

## Desafio - Tuning

Por conta do tamanho da base, seria inviálvel fazer o tuning com diversos parametros. Iremos testar apenas de diferente a parametro de perda, loss function do compile 

In [54]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.2.2', '2.16.1', '1.4.2', '0.13.0')

In [65]:
from scikeras.wrappers import KerasClassifier
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV

In [66]:
def criar_rede_tuning(loss):
    k.clear_session()
    rede_neural = Sequential([
        tf.keras.layers.InputLayer(shape=(316,)),
        tf.keras.layers.Dense(units = 158, activation = 'relu'), 
        tf.keras.layers.Dense(units = 158, activation = 'relu'), 
        tf.keras.layers.Dense(units = 1, activation = 'linear') 
    ])
    otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
    rede_neural.compile(optimizer = 'adam', loss = loss, metrics=['mean_absolute_error'])
    return rede_neural

In [67]:
rede_neural = KerasClassifier(model = criar_rede_tuning)

In [68]:
parametros = {
    'batch_size': [300],
    'epochs': [100],
    'model__loss':['mean_squared_error','mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error', 'squared_hinge']
}
parametros

{'batch_size': [300],
 'epochs': [100],
 'model__loss': ['mean_squared_error',
  'mean_absolute_error',
  'mean_absolute_percentage_error',
  'mean_squared_logarithmic_error',
  'squared_hinge']}

In [69]:
grid_search = GridSearchCV(estimator = rede_neural, param_grid = parametros, cv = 5)

In [70]:
grid_search = grid_search.fit(X,y)

/home/lucas/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Epoch 1/100


2024-11-27 14:40:53.135746: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363314048 exceeds 10% of free system memory.


959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4263724.0000
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 652888.6250
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 519355.3125
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 497849.6875
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 474875.4062
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 463385.0000
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 435029.4375
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 443088.8438
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 392081.7500
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 380310.9062
Epoch 11/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 366731.9062
Epoch 12/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 364119.0000
Epoch 13/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 355877.8438
Epoch 14/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - lo

2024-11-27 14:45:41.661028: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363315312 exceeds 10% of free system memory.


959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2261964.0000
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 625235.1250
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 531568.8750
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 500597.4062
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 489310.6875
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 457589.3125
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 437291.0938
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 442447.9688
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 416105.1562
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 403793.6562
Epoch 11/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 392953.9375
Epoch 12/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 366992.7812
Epoch 13/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 378491.7500
Epoch 14/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - lo

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



959/959 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 1.4593e-05
Epoch 29/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 7.3895e-06
Epoch 30/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 1.5800e-05
Epoch 31/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 1.4014e-05
Epoch 32/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1.3429e-05
Epoch 33/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 2.4336e-05
Epoch 43/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 9.1959e-06
Epoch 44/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 9.8327e-06
Epoch 45/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 9.1597e-06
Epoch 46/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1.9575e-05
Epoch 47/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 5.4627e-06
Epoch 48/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 5.6266e-06
Epoch 49/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 1.2673e-05
Epoch 50/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 9

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



959/959 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 2.2697e-05
Epoch 22/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 6.9651e-06
Epoch 23/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1.1866e-05
Epoch 24/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 7.9509e-06
Epoch 25/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.0642e-05
Epoch 26/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 3.3750e-06
Epoch 27/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.8697e-05
Epoch 28/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.2402e-05
Epoch 29/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.2442e-05
Epoch 30/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 2.1579e-05
Epoch 31/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 1.4069e-05
Epoch 32/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.3696e-05
Epoch 33/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 1.1106e-05
Epoch 34/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 2.

In [72]:
print(grid_search)

GridSearchCV(cv=5,
             estimator=KerasClassifier(model=<function criar_rede_tuning at 0x7e7e20d67ec0>),
             param_grid={'batch_size': [300], 'epochs': [100],
                         'model__loss': ['mean_squared_error',
                                         'mean_absolute_error',
                                         'mean_absolute_percentage_error',
                                         'mean_squared_logarithmic_error',
                                         'squared_hinge']},
             scoring='neg_mean_absolute_error')


In [73]:
melhores_parametros = grid_search.best_params_

In [74]:
print(melhores_parametros)

{'batch_size': 300, 'epochs': 100, 'model__loss': 'mean_squared_error'}


In [75]:
melhor_precisao = grid_search.best_score_
print(melhor_precisao)

-5905.833963073065
